In [1]:
import pandas as pd
import os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
import importlib

package_name = "mlflow"
try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ModuleNotFoundError:
    print(f"{package_name} not found. Installing...")
    %pip install {package_name}


mlflow is already installed.


In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-exp-2")

2023/04/07 02:49:44 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/07 02:49:44 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week2/mlruns/2', creation_time=1679093170978, experiment_id='2', last_update_time=1679093170978, lifecycle_stage='active', name='nyc-taxi-exp-2', tags={}>

In [11]:
import mlflow
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from joblib import dump, load
import tempfile
import os

models_dir = "models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

with mlflow.start_run(description="Adding Artifact as test"):

    mlflow.set_tag("developer", "bt")
    mlflow.set_tag("model", "Lasso")
    mlflow.set_tag("date", "2023-04-04")

    mlflow.log_param("train data", "/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-01.parquet")
    mlflow.log_param("test data", "/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-02.parquet")

    alpha = 0.1

    mlflow.log_param("alpha", alpha)

    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    y_pred = lasso.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)

    mlflow.log_metric("rmse", rmse)

    model_path = os.path.join(models_dir, "lasso_reg.bin")

    dump(lasso, model_path)

    mlflow.log_artifact(local_path=model_path, artifact_path="models_pickle")

    # # Save the model to a temporary file
    # with tempfile.NamedTemporaryFile(suffix=".bin", delete=False) as temp_file:
    #     dump(lasso, temp_file.name)
    #     temp_file.flush()

    #     # Log the temporary model file as an artifact
    #     mlflow.log_artifact(local_path=temp_file.name, artifact_path="models_pickle")

    # # Remove the temporary file after logging the artifact
    # os.remove(temp_file.name)


In [9]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [10]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val,label=y_val)

In [11]:
# disable auto logging

mlflow.xgboost.autolog(disable=True)

In [12]:
with mlflow.start_run(description="Saving the model with mlflow utility and preprocessor"):

    best_params =  {
        'learning_rate' : 0.0958,
        'max_depth' : 43,
        'min_child_weight' : 1.065,
        'objective' : 'reg:squarederror',
        'reg_alpha' : 0.018,
        'reg_lambda' : 0.011,
        'seed' : 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid,'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)

    rmse = mean_squared_error(y_val,y_pred,squared=False)
    mlflow.log_metric('rmse',rmse)

    with open('models/preprocessor.b','wb') as f_out:
        pickle.dump(dv,f_out)

    mlflow.log_artifact("models/preprocessor.b",artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster,artifact_path="models_pickle")

[0]	validation-rmse:13.85827
[1]	validation-rmse:12.78547
[2]	validation-rmse:11.82785
[3]	validation-rmse:10.97503
[4]	validation-rmse:10.21631
[5]	validation-rmse:9.54295
[6]	validation-rmse:8.94741
[7]	validation-rmse:8.42202
[8]	validation-rmse:7.95907
[9]	validation-rmse:7.55165
[10]	validation-rmse:7.19488
[11]	validation-rmse:6.88235
[12]	validation-rmse:6.61021
[13]	validation-rmse:6.37254
[14]	validation-rmse:6.16644
[15]	validation-rmse:5.98715
[16]	validation-rmse:5.83132
[17]	validation-rmse:5.69548
[18]	validation-rmse:5.57819
[19]	validation-rmse:5.47615
[20]	validation-rmse:5.38690
[21]	validation-rmse:5.30882
[22]	validation-rmse:5.24121
[23]	validation-rmse:5.18168
[24]	validation-rmse:5.12974
[25]	validation-rmse:5.08454
[26]	validation-rmse:5.04480
[27]	validation-rmse:5.00932
[28]	validation-rmse:4.97809
[29]	validation-rmse:4.94962
[30]	validation-rmse:4.92438
[31]	validation-rmse:4.90086
[32]	validation-rmse:4.88002
[33]	validation-rmse:4.86129
[34]	validation-rms

/home/bcthakreda/anaconda3/envs/experiment-tracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [15]:
loaded_model =  mlflow.pyfunc.load_model('runs:/cfb3df973e23455d96aba47d03cfe2b0/models_pickle')

In [18]:
xgboost_model = mlflow.xgboost.load_model('runs:/cfb3df973e23455d96aba47d03cfe2b0/models_pickle')

In [21]:
y_pred = xgboost_model.predict(valid)

In [22]:
y_pred[:10]

array([11.702003,  8.879979, 20.928928, 13.812995,  3.937456, 10.471272,
       37.448692, 21.992466, 10.64559 ,  4.125472], dtype=float32)

Define Objective Function

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model','xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid,'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val,y_pred,squared=False)
        mlflow.log_metric("rmse",rmse)
        mlflow.log_artifact(local_path='models/lin_reg.bin',artifact_path='models_pickle/')
    return {'loss': rmse, 'status': STATUS_OK}

Define Search Space

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth',4,100,1)),
    'learning_rate' : hp.loguniform('learning_rate',-3,0),
    'reg_alpha' : hp.loguniform('reg_alpha',-5,-1),
    'reg_lambda' : hp.loguniform('reg_lambda',-6,-1),
    'min_child_weight': hp.loguniform('min_child_weight',-1,3),
    'objective':'reg:linear',
    'seed': 43
}

In [24]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

ERROR [hyperopt.fmin] job exception: name 'train' is not defined


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


NameError: name 'train' is not defined

# Using hyperopt with linear regression 

In [46]:
from hyperopt import hp, tpe,fmin
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [47]:
search_space = {
    'fit_intercept': hp.choice('fit_intercept', [True, False])
}


In [48]:
best_params = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=20
)


  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

ERROR [hyperopt.fmin] job exception: 'normalize'


  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]


KeyError: 'normalize'

In [34]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'linear_regression')
        mlflow.log_params(params)

        # Create and train the linear regression model
        model = LinearRegression(fit_intercept=params['fit_intercept'])
        model.fit(X_train, y_train)

        # Make predictions on the validation set
        y_pred = model.predict(X_val)

        # Calculate the RMSE
        rmse = mean_squared_error(y_val, y_pred, squared=False)

        # Log the results with MLflow
        mlflow.log_metric("rmse", rmse)

        # Save the model
        mlflow.sklearn.log_model(model, "models_lin_reg")

    return {'loss': rmse, 'status': STATUS_OK}

best = fmin(fn=objective,space=search_space,algo=tpe.suggest,max_evals=100)

print("Best Alpha Value: ", best['alpha'])

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

ERROR [hyperopt.fmin] job exception: 'float' object has no attribute 'items'


  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]


AttributeError: 'float' object has no attribute 'items'